# Bayesian Optimization over standard imputers

In [1]:
import sys
import warnings
import pandas as pd
import numpy as np

from IPython.display import HTML, display
import tabulate

warnings.simplefilter("ignore")


from hyperimpute.utils.distributions import enable_reproducible_results

enable_reproducible_results()

# Load imputers

In [4]:
from hyperimpute.plugins.imputers import Imputers, ImputerPlugin
from hyperimpute.plugins.utils.metrics import RMSE
from hyperimpute.plugins.utils.simulate import simulate_nan
from hyperimpute.utils.optimizer import EarlyStoppingExceeded, create_study

imputers = Imputers()

imputers.list()

imputers_seed = [
    'gain',
    'softimpute',
    'sinkhorn',
    'mean', 'ice',
    'most_frequent',
    'median',
    'EM',
    'missforest',
]

subsample = 500

# Helpers

In [5]:
from sklearn.preprocessing import MinMaxScaler

def ampute(x, mechanism, p_miss):
    x_simulated = simulate_nan(x, p_miss, mechanism)
 
    mask = x_simulated["mask"]
    x_miss = x_simulated["X_incomp"]
 
    return x, x_miss, mask

def scale_data(X):
    X = np.asarray(X)
    preproc = MinMaxScaler()

    return np.asarray(preproc.fit_transform(X))

def simulate_scenarios(X):
    X = scale_data(X)
    
    datasets = {}

    mechanisms = ["MAR", "MNAR", "MCAR"]
    percentages = [0.2, 0.3] 

    for ampute_mechanism in mechanisms:
        for p_miss in percentages:        
            if ampute_mechanism not in datasets:
                datasets[ampute_mechanism] = {}
        
            datasets[ampute_mechanism][p_miss] = ampute(X, ampute_mechanism, p_miss)
            
    return datasets

# BO core

In [6]:
from typing import Any
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import time

def evaluate_plugin(
    name: str,
    plugin: ImputerPlugin,
    X: np.ndarray, 
    X_miss: np.ndarray, 
    mask: np.ndarray,
    prev_best_score: float,
):
    
    study, pruner = create_study(
        study_name=f"{name}_imputer_evaluation_{plugin.name()}",
        direction = "minimize",
        load_if_exists = False,
        patience = 5,
    )
        
    def evaluate_args(**kwargs: Any) -> float:
        imputer = plugin(**kwargs)
                
        imputed = imputer.fit_transform(X_miss.copy())
        return RMSE(np.asarray(imputed), X, mask)
        
    baseline_score = evaluate_args(**{})
    
    if baseline_score < prev_best_score:
        return baseline_score, {}
    
    pruner.report_score(baseline_score)
    if prev_best_score < 100:
        pruner.report_score(prev_best_score)
    
    def objective(trial: optuna.Trial) -> float:
        args = plugin.sample_hyperparameters(trial)
        pruner.check_trial(trial)

        score = evaluate_args(**args)

        pruner.report_score(score)

        return score

    try:
        study.optimize(objective, n_trials=50, timeout=60 * 3)
    except EarlyStoppingExceeded:
        pass
        #print(f"Early stopping triggered for imputer {plugin.name()}")
 
    if baseline_score > study.best_value:
        return baseline_score, {}
    
    return study.best_value, study.best_trial.params

def benchmark(
    name: str,
    X: np.ndarray, 
    X_miss: np.ndarray,
    mask: np.ndarray,
):
    scores = {}
    start = time.time()

    best_score = 999
    for plugin in imputers_seed:
        
        plugin_t = imputers.get_type(plugin)
        try:
            score, params = evaluate_plugin(name, plugin_t, X, X_miss, mask, best_score)
            if score < best_score:
                best_score = score
        except BaseException as e:
            print("      >>>  Plugin failed", plugin, e)
            continue
        
        scores[plugin] = (score, params)
    
    print(f" iteration for {name} took {time.time() - start} seconds")
    print(" iteration scores", scores)
    return scores
        
        

# Dataset: UCI Airfoil Self-Noise Data Set

https://archive.ics.uci.edu/ml/datasets/airfoil+self-noise


In [7]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat', header = None, sep ="\\t")

df

,0,1,2,3,4,5
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461
...,...,...,...,...,...,...
1498,2500,15.6,0.1016,39.6,0.052849,110.264
1499,3150,15.6,0.1016,39.6,0.052849,109.254
1500,4000,15.6,0.1016,39.6,0.052849,106.604
1501,5000,15.6,0.1016,39.6,0.052849,106.224


In [ ]:
frac = subsample / len(df)
X = df.sample(frac=frac).values

imputation_scenarios = simulate_scenarios(X)


results = []
candidates = {}
for scenario in ["MAR", "MCAR", "MNAR"]:
    print("Evaluating ", scenario)
    x, x_miss, mask = imputation_scenarios[scenario][0.3]

    bo_results = benchmark("airfoil", x, x_miss, mask)
    
    best_candidate = ""
    best_score = 99999
    best_params = {}
    for plugin in bo_results:
        score, params = bo_results[plugin]
        if score < best_score:
            best_score = score
            best_candidate = plugin
            best_params = params
            
    results.append([scenario, best_candidate, best_score])
    candidates[scenario] = (best_candidate, best_params)
results

headers = ["Scenario", "BO selected estimator", "BO score"]

display(
    HTML(tabulate.tabulate(results, headers=headers, tablefmt="html"))
)

In [9]:
# Full dataset evaluation
X = df.values
imputation_scenarios = simulate_scenarios(X)

results = []
for scenario in candidates:
    x, x_miss, mask = imputation_scenarios[scenario][0.3]

    plugin, plugin_params = candidates[scenario]
    
    model = imputers.get(plugin, **plugin_params)

    imputed = model.fit_transform(x_miss.copy())
    
    loss = RMSE(np.asarray(imputed), x, mask)

    results.append([scenario, plugin, loss])
    
headers = ["Scenario", "BO-selected model", "RMSE on full dataset"]


display(
    HTML(tabulate.tabulate(results, headers=headers, tablefmt="html"))
)

Scenario,BO-selected model,RMSE on full dataset
MAR,missforest,0.12575
MCAR,missforest,0.241474
MNAR,EM,0.230687


In [14]:
# Raw methods evaluation
ref_models = ["mean", "missforest", "gain", "EM", "ice", "softimpute", "sinkhorn"]
results = []
for scenario in candidates:
    x, x_miss, mask = imputation_scenarios[scenario][0.3]

    local_res = [scenario, ]
    for plugin in ref_models:
    
        model = imputers.get(plugin)

        imputed = model.fit_transform(x_miss.copy())
    
        loss = RMSE(np.asarray(imputed), x, mask)

        local_res.append(loss)
        
    results.append(local_res)
    
headers = ["Scenario"] + ref_models


display(
    HTML(tabulate.tabulate(results, headers=headers, tablefmt="html"))
)

Scenario,mean,missforest,gain,EM,ice,softimpute,sinkhorn
MAR,0.185409,0.12927,0.155535,0.136658,0.136041,0.24311,0.133777
MCAR,0.276985,0.245496,0.248649,0.232823,0.244802,0.354304,0.250449
MNAR,0.279313,0.225146,0.2742,0.230687,0.245959,0.353987,0.24993


# Dataset: Blood Transfusion Service Center Data Set


https://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center

In [15]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data', sep =",")

df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
...,...,...,...,...,...
743,23,2,500,38,0
744,21,2,500,52,0
745,23,3,750,62,0
746,39,1,250,39,0


In [ ]:
frac = min(subsample / len(df), 1)
X = df.sample(frac=frac).values

imputation_scenarios = simulate_scenarios(X)


results = []
candidates = {}
for scenario in ["MAR", "MCAR", "MNAR"]:
    print("Evaluating ", scenario)
    x, x_miss, mask = imputation_scenarios[scenario][0.3]

    bo_results = benchmark("blood", x, x_miss, mask)
    
    best_candidate = ""
    best_score = 99999
    best_params = {}
    for plugin in bo_results:
        score, params = bo_results[plugin]
        if score < best_score:
            best_score = score
            best_candidate = plugin
            best_params = params
            
    results.append([scenario, best_candidate, best_score])
    candidates[scenario] = (best_candidate, best_params)
results

headers = ["Scenario", "BO selected estimator", "BO score"]

display(
    HTML(tabulate.tabulate(results, headers=headers, tablefmt="html"))
)

In [17]:
# Full dataset evaluation
X = df.values
imputation_scenarios = simulate_scenarios(X)

results = []
for scenario in candidates:
    x, x_miss, mask = imputation_scenarios[scenario][0.3]

    plugin, plugin_params = candidates[scenario]
    
    model = imputers.get(plugin, **plugin_params)

    imputed = model.fit_transform(x_miss.copy())
    
    loss = RMSE(np.asarray(imputed), x, mask)

    results.append([scenario, plugin, loss])
    
headers = ["Scenario", "BO-selected model", "RMSE on full dataset"]


display(
    HTML(tabulate.tabulate(results, headers=headers, tablefmt="html"))
)

Scenario,BO-selected model,RMSE on full dataset
MAR,missforest,0.312208
MCAR,gain,0.241302
MNAR,missforest,0.214806


In [22]:
# Other methods evaluation

ref_models = ["mean", "missforest", "gain", "EM", "ice", "softimpute", "sinkhorn"]
results = []
for scenario in candidates:
    x, x_miss, mask = imputation_scenarios[scenario][0.3]

    local_res = [scenario, ]
    for plugin in ref_models:
    
        model = imputers.get(plugin)

        imputed = model.fit_transform(x_miss.copy())
    
        loss = RMSE(np.asarray(imputed), x, mask)

        local_res.append(loss)
        
    results.append(local_res)
    
headers = ["Scenario"] + ref_models


display(
    HTML(tabulate.tabulate(results, headers=headers, tablefmt="html"))
)

[2021-12-28T14:42:37.236094+0200][704532][CRITICAL] EM step failed. Singular matrix
[2021-12-28T14:44:04.047250+0200][704532][CRITICAL] EM step failed. Singular matrix
[2021-12-28T14:45:26.350648+0200][704532][CRITICAL] EM step failed. Singular matrix


Scenario,mean,missforest,gain,EM,ice,softimpute,sinkhorn
MAR,0.322455,0.327281,0.337027,0.292425,0.292821,0.358564,0.339201
MCAR,0.237538,0.208809,0.250994,nan,0.205612,0.247868,0.25417
MNAR,0.242604,0.215815,0.26031,nan,0.229248,0.274783,0.259288
